In [5]:
from metadata import CarPassagePaths, CarEntityPaths
from retrieval.tools import SearchTools

In [1]:
# Metadata.
datasets = {
    'entity_train': 
    ('/nfs/trec_car/data/entity_ranking/benchmarkY1_hierarchical_entity_train_data/benchmarkY1_train_entity_1000.run',
    '/nfs/trec_car/data/entity_ranking/benchmarkY1_hierarchical_entity_train_data/benchmarkY1_train_entity.qrels'),
    
    'entity_dev': 
    ('/nfs/trec_car/data/entity_ranking/benchmarkY1_hierarchical_entity_dev_data/benchmarkY1_dev_entity_1000.run',
    '/nfs/trec_car/data/entity_ranking/benchmarkY1_hierarchical_entity_dev_data/benchmarkY1_dev_entity.qrels'),
    
    'entity_test': 
    ('/nfs/trec_car/data/entity_ranking/testY1_hierarchical_entity_data/testY1_hierarchical_entity_1000.run',
    '/nfs/trec_car/data/entity_ranking/testY1_hierarchical_entity_data/testY1_hierarchical_entity.qrels'),
    
    'passage_train': 
    ('/nfs/trec_car/data/entity_ranking/benchmarkY1_hierarchical_passage_train_data/benchmarkY1_train_passage_1000.run',
    '/nfs/trec_car/data/entity_ranking/benchmarkY1_hierarchical_passage_train_data/benchmarkY1_train_passage.qrels'),
    
    'passage_dev': 
    ('/nfs/trec_car/data/entity_ranking/benchmarkY1_hierarchical_passage_dev_data/benchmarkY1_dev_passage_1000.run',
    '/nfs/trec_car/data/entity_ranking/benchmarkY1_hierarchical_passage_dev_data/benchmarkY1_dev_passage.qrels'),
    
    'passage_test': 
    ('/nfs/trec_car/data/entity_ranking/testY1_hierarchical_passage_data/testY1_hierarchical_passage_1000.run',
    '/nfs/trec_car/data/entity_ranking/testY1_hierarchical_passage_data/testY1_hierarchical_passage.qrels')
}



In [3]:
data = []
tokens = []

for dataset_name, dataset_paths in datasets.items():
    run_path = dataset_paths[0]
    qrels_path = dataset_paths[1]
    
    # Define ranking type.
    if 'passage' in dataset_name:
        index_path = CarPassagePaths.index
    elif 'entity'  in dataset_name:  
        index_path = CarEntityPaths.index
    else:
        print("NOT VALID DATASET")
    
    # Define dataset type.
    if 'train' in dataset_name:
        dataset_type = 'train'
    elif 'dev'  in dataset_name:  
         dataset_type = 'dev'
    elif 'test'  in dataset_name:  
         dataset_type = 'test'
    else:
        print("NOT VALID DATASET")
        
    search_tools = SearchTools(index_path=index_path)
    
    qrels = search_tools.retrieval_utils.get_qrels_binary_dict(qrels_path=qrels_path)
    
    # Read run files
    with open(run_path, 'r') as f:
        for line in f:
            query_encoded, _, doc_id, rank, score, _ = search_tools.retrieval_utils.unpack_run_line(line)

            # Decode query.
            try:
                query = search_tools.decode_query_car(q=query_encoded)
            except ValueError:
                print("URL utf-8 decoding did not work with Pyserini's SimpleSearcher.search()/JString: {}".format(query))
                query = search_tools.process_query_car(q=query_encoded)    
            
            # Get relevant score
            try:
                if doc_id in qrels[query_encoded]:
                    relevant = 1
                else:
                    relevant = 0
            except:
                relevant = 0
            
            # Get text.
            text = search_tools.get_contents_from_docid(doc_id=doc_id)
            if 'entity'  in dataset_name: 
                text = text.split('\n')[0]

            data.append([dataset_name, dataset_path, dataset_type, query, doc_id, rank, score, text, relevant])
            


NameError: name 'CarEntityPaths' is not defined

In [ ]:
from transformers import BertTokenizer, BertModel, BertPreTrainedModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
text = """The Philippines—was almost entirely written by a single bot. Unlike Scots speakers, Ceb"""
BERT_encodings = tokenizer.encode_plus(text=text, max_length=512, add_special_tokens=True, pad_to_max_length=True)


class BertCLS(BertPreTrainedModel):
    
    def __init__(self, config):
        super().__init__(config)
        # Initialise BERT setup.
        self.bert = BertModel(config)
        # Initialise BERT weights.
        self.init_weights()
    
    def get_BERT_cls_vector(self, 
                            input_ids, 
                            attention_mask=None, 
                            token_type_ids=None, 
                            position_ids=None, 
                            head_mask=None,
                            inputs_embeds=None):
        """ Returns BERT pooled_output (i.e. CLS vector) applying dropout. """
        # Get BERT outputs.
        outputs = self.bert(input_ids=input_ids, 
                            attention_mask=attention_mask, 
                            token_type_ids=token_type_ids,
                            position_ids=position_ids, 
                            head_mask=head_mask, 
                            inputs_embeds=inputs_embeds)
        # Apply dropout to pooled_output (i.e. CLS vector) and apply dropout.
        return outputs[1]

model = BertCLS.from_pretrained('bert-base-uncased')

input_ids = torch.tensor([BERT_encodings['input_ids']]*100)
attention_mask = torch.tensor([BERT_encodings['attention_mask']]*100)
token_type_ids = torch.tensor([BERT_encodings['token_type_ids']]*100)
import time

start = time.time()
for i in range(1):
    cls = model.get_BERT_cls_vector(input_ids=input_ids, 
                                        attention_mask=attention_mask, 
                                        token_type_ids=token_type_ids)
    
end = time.time()
print(end-start)

In [2]:
a = []
a.append([0,12,3])
a.append([4,5,6])
a

[[0, 12, 3], [4, 5, 6]]